This notebook is intended to perform some basic data cleaning before uploading to BigQuery

In [40]:
import pandas as pd
import os
from google.cloud import bigquery
import plotly.express as px

In [7]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './bq-service-account.json'

In [10]:
client = bigquery.Client(project='eg-data-assessment')

# Exercise 1

In [50]:
df = pd.read_csv('./raw_files/Exercise_1_-_Webpage_Performance.csv', thousands=',')

In [51]:
df.head()

,date,variant,metric,value
0,1/1/2024,A,visits,2115
1,1/2/2024,A,visits,5406
2,1/3/2024,A,visits,2278
3,1/4/2024,A,visits,6725
4,1/5/2024,A,visits,8920


In [52]:
df.describe()

,value
count,84.000000
mean,4660.928571
std,10821.613112
min,5.000000
25%,51.500000
50%,201.000000
75%,2890.500000
max,54944.000000


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     84 non-null     object
 1   variant  84 non-null     object
 2   metric   84 non-null     object
 3    value   84 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 2.8+ KB


In [54]:
# convert our dates
df['date'] = pd.to_datetime(df['date'])

In [55]:
# strip any unncessary spaces from our column headers
df.columns = df.columns.str.strip()

In [58]:
# pivot table to where downloads and visits are each a column. Will make additional analysis simpler
df_pivot = df.pivot_table(index=['date', 'variant'], columns='metric', values='value', aggfunc=sum).reset_index()

In [61]:
df_pivot.to_gbq('raw_data.web_performance', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


# Exercise 3

In [80]:
df_installs = pd.read_csv('./raw_files/Exercise_3_-_Installs (1).csv', thousands=',')

In [ ]:
df

In [65]:
df_installs.columns = df_installs.columns.str.strip()
df_installs.columns = df_installs.columns.str.lower()

In [68]:
df_installs['install_date'] = pd.to_datetime(df_installs['install_date'])

In [71]:
df_installs.describe()

C:\Users\Connor Luckey-Smith\AppData\Local\Temp\ipykernel_35804\22349228.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,user_id,install_date,country_code
count,3600,3600,3598
unique,3600,3536,110
top,0005be69089baac65f3dcca8bbd77f36f5fded49,2022-06-21 00:00:00,US
freq,1,7,747
first,NaN,2021-01-01 11:13:00,NaN
last,NaN,2023-12-31 22:44:00,NaN


In [72]:
df_installs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       3600 non-null   object        
 1   install_date  3600 non-null   datetime64[ns]
 2   country_code  3598 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 84.5+ KB


In [73]:
df_installs.to_gbq('raw_data.user_installs', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [85]:
df_activity = pd.read_csv('./raw_files/Exercise_3_-_Activity (1).csv')

In [86]:
df_activity.head()

,USER_ID,PLAY_DATE,MATCHES_STARTED,MATCHES_COMPLETED,MATCHES_WON
0,d4b3dccbf71946ae604d142792d1371b549a8a8e,22/08/22 00:00,5,5,0
1,950ecfee6ee85398628e33a256daccedab6c5e7e,10/02/22 00:00,15,15,0
2,950ecfee6ee85398628e33a256daccedab6c5e7e,24/07/22 00:00,15,15,2
3,9bccb7e16df4195aa74b25e8a7b88ca803d44e2a,12/09/22 00:00,4,3,0
4,01b9d4a84ad6297f78ad98fcfc82d245813332fa,25/09/21 00:00,5,5,0


In [87]:
df_activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46299 entries, 0 to 46298
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   USER_ID            46299 non-null  object
 1   PLAY_DATE          46299 non-null  object
 2   MATCHES_STARTED    46299 non-null  int64 
 3   MATCHES_COMPLETED  46299 non-null  int64 
 4   MATCHES_WON        46299 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.8+ MB


In [88]:
df_activity.columns = df_activity.columns.str.strip()
df_activity.columns = df_activity.columns.str.lower()

In [89]:
df_activity['play_date'] = pd.to_datetime(df_activity['play_date'])

In [90]:
df_activity.head()

,user_id,play_date,matches_started,matches_completed,matches_won
0,d4b3dccbf71946ae604d142792d1371b549a8a8e,2022-08-22,5,5,0
1,950ecfee6ee85398628e33a256daccedab6c5e7e,2022-10-02,15,15,0
2,950ecfee6ee85398628e33a256daccedab6c5e7e,2022-07-24,15,15,2
3,9bccb7e16df4195aa74b25e8a7b88ca803d44e2a,2022-12-09,4,3,0
4,01b9d4a84ad6297f78ad98fcfc82d245813332fa,2021-09-25,5,5,0


In [91]:
df_activity.describe()

,matches_started,matches_completed,matches_won
count,46299.000000,46299.000000,46299.000000
mean,5.607853,5.159463,0.270114
std,6.936740,6.408249,1.074767
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000
50%,3.000000,3.000000,0.000000
75%,7.000000,7.000000,0.000000
max,131.000000,80.000000,55.000000


In [94]:
df_activity.to_gbq('raw_data.user_activity', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [93]:
df_activity

,user_id,play_date,matches_started,matches_completed,matches_won
0,d4b3dccbf71946ae604d142792d1371b549a8a8e,2022-08-22,5,5,0
1,950ecfee6ee85398628e33a256daccedab6c5e7e,2022-10-02,15,15,0
2,950ecfee6ee85398628e33a256daccedab6c5e7e,2022-07-24,15,15,2
3,9bccb7e16df4195aa74b25e8a7b88ca803d44e2a,2022-12-09,4,3,0
4,01b9d4a84ad6297f78ad98fcfc82d245813332fa,2021-09-25,5,5,0
...,...,...,...,...,...
46294,2d775d90316e4c0ec139b51cb02c70b26e55a174,2024-10-03,1,1,0
46295,439e88bbfd31549fc0364f4560f8d8578d3f0f0b,2024-03-27,6,6,0
46296,bbbd705e8f063cd6e04913cc58390ac200134fe7,2024-03-27,1,1,0
46297,9142a6708661e051044b85a04b0ad7cf727b755a,2024-05-17,7,7,0
